In [ ]:
import subprocess
subprocess.run('pip install pandas pytz'.split())

In [ ]:
from os import environ
from tempfile import mkdtemp
input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/est-pst/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', mkdtemp())

In [ ]:
from os.path import join
from pandas import read_csv
timestamps_table = read_csv(join(input_folder, 'timestamps.csv'))
timestamps_table

In [ ]:
import json
settings_path = join(input_folder, 'settings.json')
settings_dictionary = json.load(open(settings_path, 'rt'))
settings_dictionary

In [ ]:
from pytz import timezone
source_tz = timezone(settings_dictionary['source_timezone'])
target_tz = timezone(settings_dictionary['target_timezone'])

In [ ]:
from datetime import datetime
for index, row in timestamps_table.iterrows():
    for raw_key, raw_value in row.items():
        raw_value = raw_value.strip()
        is_lowercase = raw_value[-1].islower()
        for timestamp_format in '%I%p', '%I:%M%p', '%Y%m%d-%H%M':
            try:
                value = datetime.strptime(raw_value, timestamp_format)
            except ValueError:
                continue
            break
        else:
            continue
        value = source_tz.localize(value).astimezone(target_tz)
        value = value.strftime(timestamp_format).lstrip('0')
        row[raw_key] = value.lower() if is_lowercase else value
timestamps_table

In [ ]:
timestamps_table.to_csv(open(join(output_folder, 'converted_timestamps.csv'), 'wt'), index=False)